# Info
: 주간 아파트 매매 가격이 변하는 경우를 8가지로 나누기 위한 코드

In [1]:
import pandas as pd
import glob

file_name = glob.glob('../article/headline_noun_keyword*')

keyword_df = []
for file in file_name:
    df = pd.read_pickle(file)
    keyword_df.append(df)

keyword_all = pd.concat(keyword_df, ignore_index = True)
keyword_all = keyword_all[keyword_all['week'] <= 807]
keyword_all

,date,co,headline,week,pos
0,2006.01.01. 오후 11:23,매일경제,취득세ㆍ등록세 0.5%P씩 인하,0,"[취득세, 인하]"
1,2006.01.01. 오후 11:23,매일경제,양도세 부담 덜자 하반기 매물 늘듯,0,"[양도세, 부담, 하반기, 매물]"
2,2006.01.01. 오후 11:23,매일경제,서울 대치동 57평형 보유세 651만원,0,"[서울, 대치동, 평, 보유, 세, 만, 원]"
3,2006.01.01. 오후 11:23,매일경제,양도세 줄이는 방법 필요경비 증빙서류 꼭 챙기자,0,"[양도세, 방법, 필요, 경비, 증빙, 서류]"
4,2006.01.01. 오후 11:23,매일경제,올해 실거래가 신고제 폭풍 이중계약서 관행 사라진다,0,"[올해, 실, 거래, 신고, 폭풍, 이중, 계약서, 관행]"
...,...,...,...,...,...
1065646,2021.06.26. 오전 5:00,뉴시스,비상 걸린 전세⑥전문가들 꼬일 대로 꼬였다…규제라도 풀어야,807,"[비상, 전세, 전문가, 대로, 규제]"
1065647,2021.06.26. 오전 3:04,세계일보,‘래미안 원베일리’ 청약에 만점 통장 등장… 올해 두 번째,807,"[래미안, 원, 베일리, 청약, 만점, 통장, 등장, 올해, 번]"
1065648,2021.06.26. 오전 3:01,동아일보,‘10억 로또 청약’ 원베일리… 7인 가족 만점통장 나왔다,807,"[억, 로또, 청약, 원, 베일리, 인, 가족, 만점, 통장]"
1065649,2021.06.26. 오전 3:01,동아일보,故 이건희 회장 이태원 주택 210억에 매물로,807,"[이건희, 회장, 이태원, 주택, 억, 매물]"


In [2]:
gu_name = ['강남구','강동구','강서구','강북구','관악구','광진구','구로구','금천구',
          '노원구','동대문구','도봉구','동작구','마포구','서대문구','성동구','성북구',
          '서초구','송파구','영등포구','용산구','양천구','은평구','종로구','중구','중랑구']

many = len(gu_name)

In [3]:
gu_data = []

for gu in gu_name:
    df = pd.read_pickle(f'../seoul_apt_trade_data/{gu}_trade.pkl')
    df = df[df['거래주'] <= 807]
    df = df.groupby(['거래주']).mean(['면적당가격'])
    for i in range(1,808):
        try:
            if df['면적당가격'][i]:
                continue
        except:
            df.loc[i] = df.loc[i-1]
    gu_data.append(df)

In [4]:
total_up_rate = []

for i in range(25):
    up_rate = gu_data[i].loc[807,'면적당가격']/gu_data[i].loc[0,'면적당가격']
    total_up_rate.append(up_rate)

In [5]:
import math

avg_list = []

for i in range(25):
    _avg = pow(10,math.log10(total_up_rate[i])/807)-1
    avg_list.append(_avg)

In [6]:
rate_variation = []

for i in range(many):
    variation = []
    for w in range(1,808):
        variation.append((gu_data[i]['면적당가격'][w] - gu_data[i]['면적당가격'][w-1])/gu_data[i]['면적당가격'][w-1])
    rate_variation.append(variation)

In [7]:
#평균보다 높게(a > avg인 a) 오른 뒤 더 큰 (b > a인 b)로 오른 경우
total_case1_idx = []
for i in range(25):
    case1_idx = []
    for w in range(1,807):
        a = rate_variation[i][w-1]
        b = rate_variation[i][w]
        if (a > avg_list[i]) and (b > a):
            case1_idx.append(w)
    total_case1_idx.append(case1_idx)

In [8]:
#평균보다 높게(a > avg인 a) 오른 뒤 더 적게 (a > b > avg인 b) 오른 경우
total_case2_idx = []
for i in range(25):
    case2_idx = []
    for w in range(1,807):
        a = rate_variation[i][w-1]
        b = rate_variation[i][w]
        if (a > avg_list[i]) and (a > b > avg_list[i]):
            case2_idx.append(w)
    total_case2_idx.append(case2_idx)

In [9]:
#평균보다 높게(a > avg인 a) 오른 뒤 평균보다 적게 (avg > b인 b) 오른 경우
total_case3_idx = []
for i in range(25):
    case3_idx = []
    for w in range(1,807):
        a = rate_variation[i][w-1]
        b = rate_variation[i][w]
        if (a > avg_list[i]) and (avg_list[i] > b):
            case3_idx.append(w)
    total_case3_idx.append(case3_idx)

In [10]:
#평균보다 적게(avg >a인 a) 오른 뒤 더 크게 (avg > b > a인 b) 오른 경우
total_case4_idx = []
for i in range(25):
    case4_idx = []
    for w in range(1,807):
        a = rate_variation[i][w-1]
        b = rate_variation[i][w]
        if (avg_list[i] > a) and (avg_list[i] > b > a):
            case4_idx.append(w)
    total_case4_idx.append(case4_idx)

In [11]:
#평균보다 적게(avg >a인 a) 오른 뒤 더 평균보다 크게 (b > avg인 b) 오른 경우
total_case5_idx = []
for i in range(25):
    case5_idx = []
    for w in range(1,807):
        a = rate_variation[i][w-1]
        b = rate_variation[i][w]
        if (avg_list[i] > a) and (b > avg_list[i]):
            case5_idx.append(w)
    total_case5_idx.append(case5_idx)

In [12]:
#평균보다 적게(avg >a인 a) 오른 뒤 더 적게 (a > b인 b) 오른 경우
total_case6_idx = []
for i in range(25):
    case6_idx = []
    for w in range(1,807):
        a = rate_variation[i][w-1]
        b = rate_variation[i][w]
        if (avg_list[i] > a) and (a > b):
            case6_idx.append(w)
    total_case6_idx.append(case6_idx)

In [16]:
#평균보다 많이 오른 주
total_case7_idx = []
for i in range(25):
    case7_idx = []
    for w in range(1,808):
        a = rate_variation[i][w-1]
        if a >= avg_list[i]:
            case7_idx.append(w)
    total_case7_idx.append(case7_idx)

In [17]:
#평균보다 적게 오른 주
total_case8_idx = []
for i in range(25):
    case8_idx = []
    for w in range(1,808):
        a = rate_variation[i][w-1]
        if a < avg_list[i]:
            case8_idx.append(w)
    total_case8_idx.append(case8_idx)

In [18]:
case_index_df = pd.DataFrame({
    'case1' : total_case1_idx,
    'case2' : total_case2_idx,
    'case3' : total_case3_idx,
    'case4' : total_case4_idx,
    'case5' : total_case5_idx,
    'case6' : total_case6_idx,
    'case7' : total_case7_idx,
    'case8' : total_case8_idx,
    'uprate': rate_variation,
    'uprate_avg' : avg_list
}, index = gu_name)
case_index_df.to_pickle('../seoul_apt_trade_data/case_index_df.pkl')

## 1. 2번째 주차의 상승률부터 고려하고있다.
## 2. case_idx에 담긴 값은 a의 index 값이므로, b = a+1이다.
## 3. 해당 주의 집값 상승률과 전 주의 빈출 키워드를 비교하거나
## 4. case에 따른 a에 해당하는 주의 빈출 키워드를 확인 해본다.